In [43]:
import numpy as np
import random
import sympy as sp 
from numpy.linalg import matrix_rank
np.set_printoptions(threshold=np.inf)

In [44]:
###Generating Ciphertexts for corresponding Plaintext with the help of Shell script file vikrant.sh

In [45]:
import os
os.system('chmod +x vikrant.sh')
os.system('./vikrant.sh')


spawn ssh student@172.27.26.188
student@172.27.26.188's password: 


		WELCOME TO THE CAVES!

		Enter your group name: SecureSavants


		Enter password: 


You have solved 5 levels so far.
Level you want to start at: 5




	You are in a passage. There is very little light here.
	You light a matchstick... the floor is full of rocks.
	A cold draught hits you from behind and you shiver. You try to
	walk quickly but stumble. Cursing your luck, you slow
	down. The passage curves downwards. You put one hand on a 
	side wall for guidance. The downward slope becomes steeper. 
	You walk very slowly to avoid falling down. The slope has 
	become very steep now... You throw away the matchstick to
	hold both side walls of the passage with your hands. It becomes
	completely dark! Your foot hits a small rock that rolls 
	down the passage. Stopping, you try to hear the sound of 
	the rock hitting the bottom and after a while you hear a 
	very distant splashing sound. The passage seems to be
	leading t

0

In [46]:
with open('vikrant_ciphertexts.txt', 'r') as file_handle:
    content = file_handle.readlines()

# Filter out empty lines and strip leading/trailing whitespaces
content = [line.strip() for line in content if line.strip()]

with open('vikrant_ciphertexts.txt', 'w') as file_handle:
    file_handle.write('\n'.join(content))


In [47]:
#MAPPINGS
map = {'f' : [0,0,0,0],
 'g' : [0,0,0,1],
 'h' : [0,0,1,0],
 'i' : [0,0,1,1],
 'j' : [0,1,0,0],
 'k' : [0,1,0,1],
 'l' : [0,1,1,0],
 'm' : [0,1,1,1],
 'n' : [1,0,0,0],
 'o' : [1,0,0,1],
 'p' : [1,0,1,0],
 'q' : [1,0,1,1],
 'r' : [1,1,0,0],
 's' : [1,1,0,1],
 't' : [1,1,1,0],
 'u' : [1,1,1,1]}

inverse_map = {'0000': 'f',
 '0001': 'g',
 '0010': 'h',
 '0011': 'i',
 '0100': 'j',
 '0101': 'k',
 '0110': 'l',
 '0111': 'm',
 '1000': 'n',
 '1001': 'o',
 '1010': 'p',
 '1011': 'q',
 '1100': 'r',
 '1101': 's',
 '1110': 't',
 '1111': 'u'}

In [48]:
def process_output(input_file, output_file):
    input_stream = open(input_file, "r")
    output_stream = open(output_file, "w+")
    
    line_count = 0
    max_lines = 8
    max_chars = 128
    
    while line_count < max_lines:
        char_count = 0
        while char_count < max_chars:
            line = input_stream.readline().split()
            output_stream.write(line[0])
            output_stream.write(" ")
            char_count += 1
        output_stream.write("\n")
        line_count += 1
    
    input_stream.close()
    output_stream.close()
# Function: decode_byte
# Description: Converts a byte to a corresponding two-character string.
def decode_byte(byte):
    binary_num = '{:0>8}'.format(format(byte,"b"))  # Convert byte to binary representation
    first_char = inverse_map[binary_num[0:4]]  # Extract first four bits and map to character
    second_char = inverse_map[binary_num[4:8]]  # Extract last four bits and map to character
    result = first_char + second_char  # Combine characters
    return result



In [49]:
process_output("vikrant_ciphertexts.txt", "vikrant_ciphertexts2.txt")

In [50]:
import numpy as np
from pyfinite import ffield
import galois

F = ffield.FField(7, gen=0x83, useLUT=-1)

In [51]:
# Maps set of two characters (bytes) to its corresponding hex value
def char_to_hex(byte):
    hex_val = chr(16*(ord(byte[0]) - ord('f')) + ord(byte[1]) - ord('f'))
    return hex_val

# Takes full block of IP e.g., 'fffffffffffffffg' and outputs corresponding hex list
def block_to_hex(block):
    hex_string = ""
    index = 0
    while index < len(block):
        hex_string += char_to_hex(block[index:index+2])
        index += 2
    return hex_string


In [52]:
# Initialization
exp_cache = [[-1]*128 for _ in range(128)]
field = ffield.FField(7)

# Functions
def custom_add(n1, n2):
    return int(n1) ^ int(n2)

def custom_multiply(n1, n2):
    return field.Multiply(n1, n2)

def custom_expo(no, pow):
    if exp_cache[no][pow] != -1:
        return exp_cache[no][pow]

    result = 0
    if pow == 0:
        result = 1
    elif pow == 1:
        result = no
    elif pow % 2 == 0:
        sqrt_no = custom_expo(no, pow >> 1)
        result = custom_multiply(sqrt_no, sqrt_no)
    else:
        sqrt_no = custom_expo(no, pow >> 1)
        result = custom_multiply(sqrt_no, sqrt_no)
        result = custom_multiply(no, result)

    exp_cache[no][pow] = result
    return result

def custom_add_vectors(v1, v2):
    result = [0] * 8
    i = 0
    while i < len(v1):
        result[i] = custom_add(v1[i], v2[i])
        i += 1
    return result

def custom_scalar_multiplication(v, elem):
    result = [0] * 8
    i = 0
    while i < len(v):
        result[i] = custom_multiply(v[i], elem)
        i += 1
    return result

def custom_linear_transformation(matrix, elements):
    result = [0] * 8
    for row, elem in zip(matrix, elements):
        result = custom_add_vectors(custom_scalar_multiplication(row, elem), result)
    return result


In [53]:

def expo(base,power):
    ans = base
    for i in range(1,power):
        ans = F.Multiply(ans,base)
    return ans


def Linear(linmat,msg):
    ans = [0]*8
    for i in range(8):
        temp  = []
        mul = [F.Multiply(linmat[i][j],msg[i]) for j in range(8)]
        for k in range(8):
            temp.append(np.bitwise_xor(ans[k],mul[k]))
        ans = temp
    return ans



def decode_block(cipher):
  plain= ""
  for i in range(0,len(cipher),2):
      plain+=chr(16*(ord(cipher[i:i+2][0]) - ord('f')) + ord(cipher[i:i+2][1]) - ord('f'))
  return plain

In [54]:
#This list will consist of all possible exponents
poss_exp = [[] for i in range(8)]
#This list will consist of all possible diagonal values
poss_aii = [[[] for i in range(8)] for j in range(8)]
input_file = open("vikrant_plaintexts.txt", 'r')
output_file = open("vikrant_ciphertexts2.txt", 'r')
for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    inpString = []
    outString = []
    #Converting each input to corresponding hex values
    for hexi in iline.strip().split(" "):
        inpString.append(block_to_hex(hexi)[ind])
    for hexo in oline.strip().split(" "):
        outString.append(block_to_hex(hexo)[ind])
        
    for i in range(1, 127):
        for j in range(1, 128):
            flag = True
            for inps, outs in zip(inpString, outString):
                #We iterate over all possible values of ei and ajj to check if input maps to output
                if ord(outs) != custom_expo(custom_multiply(custom_expo(custom_multiply(custom_expo(ord(inps), i), j), i), j), i):
                    flag = False
                    break
            if flag:
                #If yes, then we append them to corresponding possible lists
                poss_exp[ind].append(i)
                poss_aii[ind][ind].append(j)
print("Possible Diagonal Values Of Matrix A",poss_aii)
print("Possible Values Of Matrix E",poss_exp)

Possible Diagonal Values Of Matrix A [[[84, 87, 7], [], [], [], [], [], [], []], [[], [77, 70], [], [], [], [], [], []], [[], [], [108, 43, 86], [], [], [], [], []], [[], [], [], [124, 12, 88], [], [], [], []], [[], [], [], [], [5, 31, 112], [], [], []], [[], [], [], [], [], [17, 110, 11], [], []], [[], [], [], [], [], [], [123, 27, 63], []], [[], [], [], [], [], [], [], [38, 11, 58]]]
Possible Values Of Matrix E [[23, 48, 56], [20, 108], [29, 43, 55], [51, 80, 123], [78, 85, 91], [29, 43, 55], [18, 21, 88], [29, 43, 55]]


In [55]:
input_file = open("vikrant_plaintexts.txt", 'r')
output_file = open("vikrant_ciphertexts2.txt", 'r')
for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    #Considering only first 6 pairs
    if ind > 6 :
        break
    inpString = []
    outString = []
    #Converting each input to corresponding hex values
    for hexi in iline.strip().split(" "):
        inpString.append(block_to_hex(hexi)[ind]) 
    for hexo in oline.strip().split(" "):
        outString.append(block_to_hex(hexo)[ind+1])
    for i in range(1, 128):
        #We iterate over all possible pairs of exponents and diagonal values
        for p1, e1 in zip(poss_exp[ind+1], poss_aii[ind+1][ind+1]):
            for p2, e2 in zip(poss_exp[ind], poss_aii[ind][ind]):
                flag = True
                for inp, outp in zip(inpString, outString):
                    #We substitute the pairs ad=nd iterate over all possible values of i
                    #We do this by forming virtual triangle (aii,aij,ajj)
                    if ord(outp) != custom_expo(custom_add(custom_multiply(custom_expo(custom_multiply(custom_expo(ord(inp), p2), e2), p2), i) ,custom_multiply(custom_expo(custom_multiply(custom_expo(ord(inp), p2), i), p1), e1)), p1):
                        flag = False
                        break
                if flag:
                    #If we find such value, then we can discard other possibilities and finalize the lists
                    poss_exp[ind+1] = [p1]
                    poss_aii[ind+1][ind+1] = [e1]
                    poss_exp[ind] = [p2]
                    poss_aii[ind][ind] = [e2]
                    poss_aii[ind][ind+1] = [i]
print(poss_aii)
print(poss_exp)

[[[84], [119], [], [], [], [], [], []], [[], [70], [27], [], [], [], [], []], [[], [], [43], [31], [], [], [], []], [[], [], [], [12], [122], [], [], []], [[], [], [], [], [112], [98], [], []], [[], [], [], [], [], [11], [95], []], [[], [], [], [], [], [], [27], [4]], [[], [], [], [], [], [], [], [38]]]
[[23], [108], [43], [80], [91], [55], [21], [29]]


In [56]:
def EAEAE (msg, lin_mat, exp_mat):
  msg = [ord(m) for m in msg]
  res = [expo(msg[i], exp_mat[i]) for i in range(8)]
  res = Linear(lin_mat, res)
  res = [expo(res[i], exp_mat[i]) for i in range(8)]
  res = Linear(lin_mat, res)
  res = [expo(res[i], exp_mat[i]) for i in range(8)]
  return res
#This function mimics EAEAE
def EAEAE_half (plaintext, lin_mat, exp_mat):
    plaintext = [ord(c) for c in plaintext]
    Output = [[0 for j in range (8)] for i in range(8)]
    for ind, elem in enumerate(plaintext):
        Output[0][ind] = custom_expo(elem, exp_mat[ind])

    Output[1] = custom_linear_transformation(lin_mat, Output[0])

    for ind, elem in enumerate(Output[1]):
        Output[2][ind] = custom_expo(elem, exp_mat[ind])

    Output[3] = custom_linear_transformation(lin_mat, Output[2])

    for ind, elem in enumerate(Output[3]):
        Output[4][ind] = custom_expo(elem, exp_mat[ind])
    return Output[4]
def arange_linear_matrix(matrix):
    # Get the dimensions of the matrix
    rows = len(matrix)
    cols = len(matrix[0])

    # Create a new matrix to store the transpose
    transpose = [[0 for _ in range(rows)] for _ in range(cols)]

    # Iterate through the original matrix and fill in the transpose matrix
    for i in range(rows):
        for j in range(cols):
            transpose[j][i] = matrix[i][j]

    return transpose

In [57]:
input_file = open('vikrant_plaintexts.txt','r')
output_file = open('vikrant_ciphertexts2.txt','r')
input = input_file.readlines()
output = output_file.readlines()

In [58]:

in_string = []
for i in range(len(input)):
    x = []
    for msg in input[i].strip().split(' '):
        x.append(decode_block(msg))
    in_string.append(x)


out_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg))
    out_string.append(x)

for indexex in range(0,6):
    offset = indexex + 2
    
    exp_list = [e[0] for e in poss_exp]
    lin_tran_lst = np.zeros((8,8),int)

    for i in range(8):
        for j in range(8):     
            if(len(poss_aii[i][j]) != 0):
                lin_tran_lst[i][j] = poss_aii[i][j][0]
            else:
                lin_tran_lst[i][j] = 0
    
    for index in range(8):
        if(index > (7-offset)):
            continue

        for i in range(127):
            lin_tran_lst[index][index+offset] = i+1
            flag = True
            for inps, outs in zip(in_string[index], out_string[index]):
                x1 = EAEAE(inps, lin_tran_lst, exp_list)[index+offset]
                x2 = outs[index+offset]
                if x1 != ord(x2):
                    flag = False
                    break
            if flag==True:
                poss_aii[index][index+offset] = [i+1]
                
A = np.zeros((8,8),dtype='int')

for i in range(0,8):
    for j in range(0,8):
        if len(poss_aii[j][i]) != 0:
            A[i][j] = poss_aii[j][i][0]



E = exp_list

print('\n\nLinear Transformation Matrix :\n',A)
print('\n\n')
print('Exponent Vector : \n',E)



Linear Transformation Matrix :
 [[ 84   0   0   0   0   0   0   0]
 [119  70   0   0   0   0   0   0]
 [ 20  27  43   0   0   0   0   0]
 [125  23  31  12   0   0   0   0]
 [ 96  57   7 122 112   0   0   0]
 [ 29  40  24  45  98  11   0   0]
 [ 23 124  14  98   2  95  27   0]
 [ 95  12  77  29  24  70   4  38]]



Exponent Vector : 
 [23, 108, 43, 80, 91, 55, 21, 29]


In [59]:
Einverse = np.zeros((128, 128), dtype = int)

for base in range(0,128):
    temp = 1
    for power in range(1,127):
        result = F.Multiply(temp, base)
        Einverse[power][result] = base
        temp = result
    
GF = galois.GF(2**7)
A = GF(A)
invA = np.linalg.inv(A)
password = "mshsmqhhjikpmflrmngkkqkpikkgjkjq"
GF = galois.GF(2**7)

def Einv(block, E):
    return [Einverse[E[i]][block[i]] for i in range(8) ]

def Ainv(block, A):
    block = GF(block)
    A = GF(A)
    return np.matmul(A,block)

decrypted_password = ""
for i in range(0,2): 
    elements = password[16*i:16*(i+1)]
    currentBlock = []
    for j in range(0,15,2):
        currentBlock+=[(ord(elements[j]) - ord('f'))*16 + (ord(elements[j+1]) - ord('f'))]
    EAEAE = Einv(Ainv(Einv(Ainv(Einv(currentBlock, E), invA),E), invA),E)
    for ch in EAEAE:
        decrypted_password += chr(ch)
    
print("\n\nPassword is",decrypted_password)



Password is wmtvcbnzha000000


In [60]:
#Two halves of password
password_1 = "mshsmqhhjikpmflr"
password_2 = "mngkkqkpikkgjkjq"

T_A=arange_linear_matrix(A)

#We iterate over all possible 128 byte values and perform EAEAE to check for which input the password (half) matches
def DecryptPassword(password):
    passw = block_to_hex(password)
    op = ""
    for ind in range(8):
        for ans in range(128):
            inp = op + decode_byte(ans)+(16-len(op)-2)*'f'
            if ord(passw[ind]) == EAEAE_half(block_to_hex(inp), T_A, E)[ind]:
                op += decode_byte(ans)
                break
    return op

print(block_to_hex(DecryptPassword(password_1))+block_to_hex(DecryptPassword(password_2)))

wmtvcbnzha000000


In [61]:
def ascii_values(string):
    ascii_list = []
    for char in string:
        ascii_value = ord(char)
        ascii_list.append((ascii_value))
    return ascii_list



print("Decrypted Block 1 ASCII:",ascii_values(block_to_hex(DecryptPassword(password_1))))
print("Decrypted Block 2 ASCII:",ascii_values(block_to_hex(DecryptPassword(password_2))))

Decrypted Block 1 ASCII: [119, 109, 116, 118, 99, 98, 110, 122]
Decrypted Block 2 ASCII: [104, 97, 48, 48, 48, 48, 48, 48]


In [62]:
print(block_to_hex(DecryptPassword(password_1)))
print(block_to_hex(DecryptPassword(password_2)))

wmtvcbnz
ha000000
